In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

In [2]:
import pandas as pd

df = pd.read_csv(r'c:\Carrer\Data_science\Thesis_bachelor\Data\preprocess.csv')

In [1]:
tokenizer_gemma = AutoTokenizer.from_pretrained("ModelSpace/GemmaX2-28-2B-v0.1")
model_gemma = AutoModelForCausalLM.from_pretrained("ModelSpace/GemmaX2-28-2B-v0.1")

NameError: name 'AutoTokenizer' is not defined

In [3]:
df

,Thai,Viet
0,- รร.แกรนด์โฮเต็ล,- Khách sạn Grand.
1,- สายไม่ว่าง,- Đường dây bận.
2,นั่นใครพูดคะ,Ai đấy?
3,ฉันจะโอนสายคุณ ไปที่แผนกรูมเซอร์วิส,Tôi có thể kết nối quý khách với dịch vụ phòng.
4,ฮัลโหลๆ,A lô.
...,...,...
15193987,ทำไมพระเจ้าถึงมอบความสามารถนี้ให้กับเรา,Tại sao Chúa lại ban cho bọn cháu khả năng này?
15193988,ไม่ว่าพระเจ้าจะให้อะไรเรามา จะดีหรือร้าย เราก็...,"Dù Chúa ban gì cho ta, dù tốt hay xấu, thì ta ..."
15193989,เพราะมัน เราจึงได้มีชีวิตอย่างไร้ซึ่งสิ่งติดค้าง,"Vì có nó, ta có thể sống một cuộc sống không c..."
15193990,พวกเราที่ได้เปิดดวงตาที่สามนั้น มีชีวิตอยู่ก็เ...,Những người đã mở con mắt thứ ba như chúng ta ...


In [4]:
src_texts = df['Thai'].tolist()
tgt_texts = df['Viet'].tolist()

In [9]:
from tqdm import tqdm
import torch
import random

# Your existing model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

# Define languages
src_lang = "tha_Thai"  # Thai
tgt_lang = "vie_Latn"  # Vietnamese

# Move model to appropriate device if not already done
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Set source language
tokenizer.src_lang = src_lang

# Assuming src_texts is your full dataset
# src_texts = ["สวัสดี", "ขอบคุณ", ...]  # Your full list of Thai texts

# Take a random sample of 1000 lines
src_texts_subset = random.sample(src_texts, 5000)  # Randomly select 1000 lines

predictions = []
for src in tqdm(src_texts_subset):  # Use the random subset
    # Tokenize input and move to same device as model
    inputs = tokenizer(src, return_tensors="pt", truncation=True).to(model.device)
    
    # Generate translation with target language code
    generated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_length=100,
        num_beams=5
    )
    
    # Decode prediction
    pred = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    predictions.append(pred)

# Optional: Print results for the random subset
for src, pred in zip(src_texts_subset, predictions):
    print(f"Source: {src}")
    print(f"Translation: {pred}\n")

100%|██████████| 5000/5000 [4:02:11<00:00,  2.91s/it]     


Source: เขาเป็นคนแบบนี้ล่ะ เบื้องหลังเขาไม่ใช่เล่นๆ เลย
Translation: Anh ta là người như thế này.

Source: - ทิ้งไว้ - ผมว่าแบบนั้นไม่ดีมั้ง
Translation: - Tôi không nghĩ thế là tốt.

Source: อะไรอยู่ในนี้วะเนี่ย
Translation: Cái quái gì ở đây?

Source: ขอโทษ คุณตำรวจเฮอแนนเดซ
Translation: Tôi xin lỗi, Cảnh sát Hernandez.

Source: ขอโทดเรื่องคอกไม้นะค่ะ ผมโอเค ผมไม่เป็นไร
Translation: Tôi không sao. Tôi không sao.

Source: - แจ้งที่เขตไว้ด้วย
Translation: - Đăng ký ở quận.

Source: ยังไงเราก็พังดิอาร์เคเดียนอยู่ดี
Translation: Dù sao thì chúng ta cũng đang ở trong vùng Arcadia.

Source: ‎ขอโทษที
Translation: Tôi xin lỗi.

Source: แค่เพราะเราส่งเสียงดังได้ ไม่ใช่ว่าเราต้องส่งเสียงดัง
Translation: Chỉ vì chúng ta có thể phát ra tiếng ồn, không phải vì chúng ta phải phát ra tiếng ồn.

Source: ‎แล้วเจ้าจะไปยังไง เจ้าไม่มีเหรียญแล้ว
Translation: Vậy anh sẽ đi đâu? Anh không có xu nữa.

Source: ใกล้ถึงแล้ว
Translation: Gần rồi.

Source: ฉันกำลังไป แต่มันไกลกว่าที่คิดน่ะ
Translation: Tôi đang

In [12]:
import sacrebleu

# Assuming you have reference translations in a list called ref_texts
# ref_texts = ["Xin chào", "Cảm ơn", ...]  # Full list matching src_texts

# Get the indices of the sampled subset
# Since src_texts_subset was randomly sampled, we need to match it back to ref_texts
# If you didn't store the indices, we’ll need to find them (see below)
# For now, assuming ref_texts_subset is available or can be derived

# If you have ref_texts as a list matching the full src_texts:
# Find the indices of src_texts_subset in the original src_texts
sampled_indices = [src_texts.index(src) for src in src_texts_subset]
ref_texts_subset = [ref_texts[i] for i in sampled_indices]

# Compute BLEU score
bleu = sacrebleu.corpus_bleu(predictions, [ref_texts_subset])

print(f"BLEU score: {bleu.score:.2f}")

# Optional: Detailed output
print(f"Detailed BLEU: {bleu}")

KeyboardInterrupt: 